# CS-5313/7313 Project 4

## IMPORTANT - Make a copy of this colab notebook before working on it.

In this project, you will be exploring various pre-built language models based on the transformer architecture. Transformer networks are a state of the art approach to langauge and time series modeling that makes use a concept called "attention". The first time this design was proposed was in the paper "Attention is All You Need" by Vaswani et al. The paper can be read here: https://arxiv.org/pdf/1706.03762.pdf

Here you will be making use of the pre-built transformer pipelines provided by Hugging Face Co. You can reference this link on how to use the package for the given task you are trying to complete: https://huggingface.co/transformers/task_summary.html

In [ ]:
# Run me to install the package we will be using

! pip install transformers datasets
from transformers import pipeline

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 290 kB 31.8 MB/s 
     |████████████████████████████████| 596 kB 42.1 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
     |████████████████████████████████| 895 kB 54.3 MB/s 
     |████████████████████████████████| 59 kB 5.7 MB/s 
     |████████████████████████████████| 243 kB 45.3 MB/s 
     |████████████████████████████████| 132 kB 48.8 MB/s 
     |████████████████████████████████| 1.1 MB 66.7 MB/s 
     |████████████████████████████████| 271 kB 51.2 MB/s 
     |████████████████████████████████| 192 kB 49.4 MB/s 
     |████████████████████████████████| 160 kB 71.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Task 1 - Text generation

In this task you will be using the "text-generation" pipeline to generate text.

Use three different sized prompts, 1 word, ~5 words, and ~10 words, to generate sequences of length n+5 words, n+20 words, and 100 words, where n is the number of words in the prompt phrase you provided. Generate 3 sequences for each prompt and output length pair. Since this is qualitative, comment on the relative quality of the text that is generated in your report and include examples. How do the parameters affect the quality of the output. In addition to the report, submit another document containing each of these generated sequences, including what the prompt was.

In [ ]:
# TODO Task 1

## Task 2 - Sentiment Analysis

Here you will be using the "sentiment-analysis" pipeline to look at the sentiment of Amazon reviews. The database is provided at this link: https://jmcauley.ucsd.edu/data/amazon/ . It is recommended to use one of the smaller databases, such as the Musical Instruments database with 10,261 reviews.

Each review has both the text of the review, as well as the reviewer's rating.

### Subtask 2.1
Perform sentiment analysis on each review, and compare the model's output to the users review to get a sense of the accuracy of the model. The user review score, which is out of a maximum 5 stars, is found in the "overall" datafield. For this, assume that 3+ in the "overall" datafield is a positive review.

### Subtast 2.2
In addition to looking at the accuracy of the model for each review, also compare the percentage of products with more positive reviews than negative reviews to the true percentage.

Reminder: Refer to the hugging face link on how to perform sentiment analysis task.


In [ ]:
# TODO Task 2

## Task 3 - Masked Language Modeling

In this task you will be completing a sentence or phrase that has a missing word. I will prepare three datasets for you so you can perform this task. The three datasets will contain missing verbs in one, missing nouns in another, and missing adjectives in the last.

Your task will be to look at the success rate of generating the true missing word in the top 1, top 5, and top 10 generated words for a given sentence. You will then compare the success rate between the three datasets.

At the moment, these datasets are not prepared. I will update this project and send out and email later so you can download the datasets.

In [ ]:
# TODO Task 3